### Q1. What's the version that you have?

In [1]:
! mlflow --version

mlflow, version 2.3.2


### Q2. So what's the size of the saved DictVectorizer file?

We'll use the Green Taxi Trip Records dataset to predict the amount of tips for each trip.

Download the data for January, February and March 2022 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script preprocess_data.py.

The script will:

- load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
- fit a DictVectorizer on the training set (January 2022 data),
- save the preprocessed datasets and the DictVectorizer to disk.

Your task is to download the datasets and then execute this command:

```python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output```


In [ ]:
! cd data && curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
! cd data && curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet
! cd data && curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet

In [4]:
! python ./scripts/preprocess_data.py --raw_data_path "data/" --dest_path ./output

In [7]:
import os

def printSize(fileAbsPath):
  print (os.path.getsize(fileAbsPath) // 1024, "KB")

printSize(os.path.abspath(os.path.join("output", "dv.pkl")))

150 KB


### Q3. Train a model with autolog

We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework.

The script will:

    - load the datasets produced by the previous step,
    - train the model on the training set,
    - calculate the RMSE score on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with `mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

**What is the value of the max_depth parameter?**

In [9]:
! python ./scripts/train.py

2023/06/04 21:15:40 INFO mlflow.tracking.fluent: Experiment with name 'module-2-hw' does not exist. Creating a new experiment.
2023/06/04 21:15:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\Dev\projects\MLOpsZoomcamp\Module2\.venv\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first."


### Q4. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the RandomForestRegressor using optuna. We have prepared the script hpo.py for this exercise.

Your task is to modify the script hpo.py and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

**Note: Don't use autologging for this exercise.**

The idea is to just log the information that you need to answer the question below, including:

    - the list of hyperparameters that are passed to the objective function during the optimization,
    - the RMSE obtained on the validation set (February 2022 data).

**What's the best validation RMSE that you got?**

In [14]:
! python ./scripts/hop.py

2023/06/04 21:49:22 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
[I 2023-06-04 21:49:23,126] A new study created in memory with name: no-name-2e680ea8-c06b-4dce-87ec-e0e8171e200a
[I 2023-06-04 21:49:27,277] Trial 0 finished with value: 2.451379690825458 and parameters: {'n_estimators': 25, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-04 21:49:28,347] Trial 1 finished with value: 2.4667366020368333 and parameters: {'n_estimators': 16, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-04 21:49:32,377] Trial 2 finished with value: 2.449827329704216 and parameters: {'n_estimators': 34, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 2 with value: 2.449827329704216.
[I 2023-06-04 21:49:34,276] Trial 3 finished with value: 2.460983516558473 a

### Q5. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. 

We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2022 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tips for MLflow:

    - you can use the method search_runs from the MlflowClient to get the model with the lowest RMSE,
    - to register the model you can use the method mlflow.register_model and you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", and the name of the model (make sure to choose a good one!).

**What is the test RMSE of the best model?**

In [36]:
! python ./scripts/register_model.py

Test RMSE of best run: 2.2854691906481364


2023/06/04 23:48:46 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\Dev\projects\MLOpsZoomcamp\Module2\.venv\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first."
Successfully registered model 'random-forest-best-models'.
2023/06/04 23:49:25 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-best-models, version 1
Created version '1' of model 'random-forest-best-models'.
